In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
import math
import pathlib
import pandas as pd

#from sklearn.model_selection import KFold, StratifiedKFold

#from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
dataset_dir = '/kaggle/input/dataset-foglie/training'

labels = ['Apple','Blueberry','Cherry','Corn','Grape','Orange','Peach','Pepper','Potato','Raspberry','Soybean','Squash','Strawberry','Tomato']

#%%script false --no-raise-error
num_row = len(labels)//2
num_col = len(labels)//num_row
fig, axes = plt.subplots(num_row, num_col, figsize=(2*num_row,15*num_col))
for i in range(len(labels)):
  if i < len(labels):
    class_imgs = next(os.walk('{}/{}/'.format(dataset_dir, labels[i])))[2]
    class_img = class_imgs[0]
    img = Image.open('{}/{}/{}'.format(dataset_dir, labels[i], class_img))
    ax = axes[i//num_col, i%num_col]
    ax.imshow(np.array(img))
    ax.set_title('{}'.format(labels[i]))

In [ ]:
SEED = 19011997
SPLIT = 0.2
BATCH_SIZE = 16
IMG_SIZE = (256,256)

In [ ]:
dataset_dir_path = pathlib.Path(dataset_dir)
images_count_total = len(list(dataset_dir_path.glob('*/*.jpg')))
print('Total count: {}'.format(images_count_total))
images_count = {}
for label in labels:
    count = len(list(dataset_dir_path.glob('{}/*.jpg'.format(label))))
    images_count[label] = count
    print('{} count: {}'.format(label, count))

In [ ]:
max_class = max(images_count.values())
class_weights = {}
i = 0
for k,v in images_count.items():
    class_weights[i] = max_class/v
    i += 1
print(class_weights)

In [ ]:
def import_dataset(subset):
    return tf.keras.preprocessing.image_dataset_from_directory(
        dataset_dir,
        labels='inferred',
        label_mode='categorical',
        class_names=labels,
        color_mode='rgb',
        batch_size=BATCH_SIZE,
        image_size=IMG_SIZE,
        shuffle=True,
        seed=SEED,
        validation_split=SPLIT,
        subset=subset,
        interpolation='bilinear',
        follow_links=False,
        crop_to_aspect_ratio=False
    )

dataset_training = import_dataset('training')
dataset_validation = import_dataset('validation')

In [ ]:
data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.RandomFlip("horizontal_and_vertical"),
        tf.keras.layers.RandomRotation(0.4, fill_mode='constant'), # oppure "nearest"
        tf.keras.layers.RandomZoom(
    height_factor=(-0.1,0.1), width_factor=(-0.1,0.1), fill_mode='constant',
    interpolation='bilinear', seed=None, fill_value=0.0),
        tf.keras.layers.RandomContrast(factor=0.3, seed=None)
    ]
)

plt.figure(figsize=(10, 10))
for images, _ in dataset_training.take(1):
  for i in range(6):
    augim = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augim[i].numpy().astype("uint8"))
    plt.axis("off")

In [ ]:
def make_base(input_Shape, trainable=False):
    base_model = tf.keras.applications.Xception(
        weights='imagenet',
        input_shape=input_Shape,
        include_top=False
    )

    base_model.trainable = trainable

    return base_model

def make_model(input_shape, num_classes, trainable=False):
    base_model = make_base(input_shape, trainable)
    inputs = tf.keras.Input(shape=input_shape)
    x = data_augmentation(inputs)
    x = tf.keras.layers.Rescaling(1./255)(x)
    x = base_model(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(num_classes*2, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    return tf.keras.Model(inputs, outputs)

model = make_model(IMG_SIZE + (3,), len(labels))
model.summary()

In [ ]:
EPOCHS = 60

patience = 15
callbacks = [
    tf.keras.callbacks.ModelCheckpoint("./ckp_best.h5", save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=patience),
]
#model = tf.keras.models.load_model('/kaggle/input/checkp2/ckp_best2.h5')
model.compile(
    optimizer="Nadam",
    loss="categorical_crossentropy",
    metrics=["accuracy","mse"],
)
model.fit(
    dataset_training, epochs=EPOCHS, callbacks=callbacks, validation_data=dataset_validation, class_weight=class_weights

)

In [ ]:
model.save("./xception.h5")

In [ ]:
model_xc= tf.keras.models.load_model('./ckp_best.h5')

In [ ]:
#model_xc.save_weights('./weights')

In [ ]:
model_xc = make_model(IMG_SIZE + (3,), len(labels), True)
model_xc.summary()

In [ ]:
model_xc.load_weights('/kaggle/input/weights2/weights')

In [ ]:
model_xc.trainable=True
model_xc.summary()

In [ ]:
model_xc.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy", "mse"],
)
model_xc.evaluate(dataset_validation)

In [ ]:


model_xc.fit(
    dataset_training,
    validation_data=dataset_validation,
    epochs=8,
    class_weight=class_weights
)

In [ ]:
model_xc.save("./try_fine.h5")